**test**

In [1]:
import plotly.plotly as plot
import plotly.graph_objs as go
import pandas as pd

Let's load some expression data from InterMine

In [2]:
from intermine.webservice import Service
service = Service("http://www.humanmine.org/humanmine/service")

query = service.new_query("AtlasExpression")
query.add_view(
    "condition", "expression", "pValue", "tStatistic", "dataSets.name"
)

Give the query constraints

In [3]:
geneToQuery = "GATA1"

query.add_constraint("pValue", "<=", "0.005")
query.add_constraint("gene.symbol", "=", geneToQuery)

<BinaryConstraint: AtlasExpression.gene.symbol = GATA1>

This is good, but we can only iterate the data (like below). We need it in a dataframe for plotly. 
``` python
for row in query.rows(start=0,size=10):
    print (row["condition"], row["gene.primaryIdentifier"], row["gene.symbol"], row["gene.name"], 
        row["expression"], row["pValue"], row["tStatistic"], row["dataSets.name"])
```

Load iterator into data frame by popping it in a dict first (thanks nupur)

In [23]:
#Get data in the right shape
upRegulated = {}
downRegulated = {}
for row in query.rows(start=0,size=30):
    rowDict = row.to_d()
    if (row["expression"] == "UP"):
        upRegulated.update({row["condition"] : rowDict})
    else:
        downRegulated.update({row["condition"] : rowDict})


#Now pop it into the data frame
dfUp = pd.DataFrame(data=upRegulated)
dfDown = pd.DataFrame(data=downRegulated)

#The columns were the wrong way around
dfUp = dfUp.transpose()
dfDown = dfDown.transpose()

len(dfUp)
len(dfDown)

13

Now, let's use plotly to make a graph

In [60]:
#We need to define the up and down expression separately if we want to colour them separately
up = go.Bar(y=dfUp["AtlasExpression.condition"],
            x=dfUp["AtlasExpression.tStatistic"],
            name = "Upregulated",
            orientation = "h")

down = go.Bar(y=dfDown["AtlasExpression.condition"],
              x=dfDown["AtlasExpression.tStatistic"],
              name= "Downregulated",
              orientation = "h")

layout = go.Layout(
    title = "AtlasExpression data for " + geneToQuery,
    #dtick is important to make sure no labels get hidden
    #showgrid gives us the lines so we can match label to bar
    yaxis = dict(dtick = 1,
                 showgrid = True),
    #Wide margin so the labels don't get too truncated.
    margin = dict(l = 200))

fig = go.Figure(data = [up, down], layout=layout)

#Render the plot! Note for this to work on your local machine,
# You'll need a token set up as per https://plot.ly/python/getting-started/ 
plot.iplot(fig, filename='basic_bar')